# Compare LLama Providers

Run evaluations on a few prompts for llama3.1 70B across several providers and comprate results to baseline.

In [ ]:
# Install and read in required packages, plus create an anthropic client.
print('⏳ Installing packages')
%pip install -q weave set-env-colab-kaggle-dotenv tqdm ipywidgets requests groq together
print('✅ Packages installed')

In [ ]:
from tqdm.notebook import tqdm_notebook as tqdm
from set_env import set_env
from openai import OpenAI
from groq import Groq
from together import Together
import weave
import os
import json
import requests
set_env("OPENROUTER_API_KEY")
set_env("GROQ_API_KEY")
set_env("WANDB_API_KEY")

groqclient = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
client = Together(api_key=os.environ.get('TOGETHER_API_KEY'))

In [ ]:
# Initialize Weave project
weave.init('compare-llamas')


class LlamaModel(weave.Model):
    provider: str

    @weave.op()
    def predict(self, prompt: str) -> str:
        data = {
            "model": "meta-llama/llama-3.1-70b-instruct",
            "messages": [
                {"role": "user", "content": prompt}
            ],
            "temperature": 0,
            "provider": {
                "order": [self.provider],
                "allow_fallbacks": False
            }
        }
        
        response = self.make_openrouter_request(data)
        return response['choices'][0]['message']['content']

    @weave.op()
    def make_openrouter_request(self, data):
        try:
            response = requests.post(
                "https://openrouter.ai/api/v1/chat/completions",
                headers = {
                    "Authorization": f"Bearer {os.environ['OPENROUTER_API_KEY']}",
                    "Content-Type": "application/json"
                },
                json=data
            )
            response.raise_for_status()  # Raises an HTTPError for bad responses
            return response.json()
        except requests.RequestException as e:
            raise Exception(f"API request failed: {str(e)}")

class GroqModel(weave.Model):

    @weave.op()
    def predict(self, prompt: str) -> str:
        response = groqclient.chat.completions.create(
            model='llama-3.1-70b-versatile',
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            temperature=0.0,
            seed=123123
        )
        return response.choices[0].message.content
    
class TogetherModel(weave.Model):
    @weave.op()
    def predict(self, prompt: str) -> str:
        response = self.make_together_request(prompt)
        return response.choices[0].message.content

    @weave.op()
    def make_together_request(self, prompt):
        response = client.chat.completions.create(
            model="meta-llama/Meta-Llama-3.1-70B-Instruct-Turbo",
            messages=[
                {
                    "role": "user",
                    "content": prompt
                }
            ],
            temperature=0,
        )
        return response

# Create LLamas per provider OctoAI, NovitaAI, DeepInfra, Together, Fireworks
octoai_llama = LlamaModel(provider='OctoAI', name='OctoAILLa_LLama3.1_70B')
novitaai_llama = LlamaModel(provider='Novita', name='NovitaAI_LLaMa3.1_70B')
deepinfra_llama = LlamaModel(provider='DeepInfra', name='DeepInfra_LLaMa3.1_70B')

fireworks_llama = LlamaModel(provider='Fireworks', name='Fireworks_LLaMa3.1_70B')
groq_llama = GroqModel(name='Groq_LLaMa3.1_70B')
together_llama = TogetherModel(name='Together_LLaMa3.1_70B')

print("✅ Weave models created")

In [ ]:
# Let's build a dataset of quirky prompts and potentially their answers 
from weave import Dataset

quirky_prompts = Dataset(
    name="my_llama_quirky_prompts",
    rows=[
        {
            "question": "Give me 10 sentences that end in the word \"apple\"",
            "rubric": "all sentences must end with the word apple"
        },
        {
            "question": "Answer with the number of legs about the following statement: The fox lost a leg, but then magically grew back the leg he lost and a mysterious extra leg on top of that",
            "rubric": "Answer must be 5 or five"
        },
        {
            "question": "Yam (a boy) has 4 sisters. Each sister has 3 brothers. How many brothers does Yam have? Let's think step by step.",
            "rubric": "Answer must indicate that Yam has 2 brothers"
        },
        {
            "question": "You have five apples today, you ate two apples yesterday so how many apples do you have today? Provide a logical answer.",
            "rubric": "Answer must be five and explain that yesterdays action have no bearing on todays apple quantity"
        },
        {
            "question": "Which number is bigger: 9.11 or 9.9?",
            "rubric": "Answer should conclude that 9.9 is bigger" 
        },
        {
            "question": "If I hang 5 shirts outside and it takes them 5 hours to dry, how long would it take to dry 30 shirts",
            "rubric": "Answer must state that it would take the same amount of time"
        },
        {
            "question": "There are three sisters in a room. Anna is reading a book. Alice is playing a match of chess against someone in the room. What is the third sister, Amanda, doing?",
            "rubric": "Playing chess with Alice"
        },
        {
            "question": """Determine all triples (x, y, z) of real numbers that are solutions to the following
system of equations:
log9 x + 10g9 y + 10g3 z = 2
log 16 x + log4 y + log16 z = 1
log5 x + log25 y + log25 z = 0
""",
            "rubric": "IDK the answer to this one"
        }
    ]
)


In [ ]:
# Get all instantiated models
# models = [octoai_llama, together_llama]
models = [deepinfra_llama, fireworks_llama, groq_llama]


#define our scoring functions
@weave.op()
def has_response(rubric: str, model_output: dict) -> dict:
    return {'has_response': model_output is not None}

# Define the preprocess_model_input function
def preprocess_model_input(row):
    return {'prompt': row['question']}


# Define the evaluation
evaluation = weave.Evaluation(
    name='quirky_prompts_eval',
    dataset=quirky_prompts,
    trials=1,
    scorers=[
        has_response
    ],
    preprocess_model_input=preprocess_model_input
)

# Run evaluation for each model
results = {}
for model in models:
    print(f"Evaluating {model.name}...")
    result = await evaluation.evaluate(model)
    results[model.name] = result
